<font color='green'>
    <h1 align=center>Mesure de la puissance des forêts aléatoires</h1>
        <font color='black'><h3 align=center><i>From Kaggle : <a href='https://www.kaggle.com/uciml/human-activity-recognition-with-smartphones'>Human Activity Recognition Using Smartphones Data Set</a></i> </h3></font>
    </font>

<h2 align=center>CentraleSupélec-OpenClassrooms Certifiyng Training</h2>

<h3 align=center>Jean Martial Tagro </h3>

Dans ce chapitre, on va appliquer l’algorithme des forêts aléatoires sur un sur des données réelles : le <a href='https://www.kaggle.com/uciml/human-activity-recognition-with-smartphones'>Human Activity Recognition Using Smartphones Data Set</a>. Il permet de reconnaître l’activité physique à partir de données du smartphone. Il est simple mais possède de nombreuses variables (> 500) ce qui va nous permettre d’étudier un certain nombre de choses.Dans ce chapitre, on va appliquer l’algorithme des forêts aléatoires sur un sur des données réelles : le <a href=''></a>. Il permet de reconnaître l’activité physique à partir de données du smartphone. Il est simple mais possède de nombreuses variables (> 500) ce qui va nous permettre d’étudier un certain nombre de choses.

Ce jeu de données contient les logs de capteurs de smartphone sur une trentaine d'individus en train d'effectuer des activités (s'assoir, se mettre debout, marcher, etc). L'objectif sera de prédire à partir des logs de capteurs le type d'activités que le sujet est en train d'effectuer.

#### Axes d'étude
En regardant le fichier de description du dataset, on peut observer qu'il y a beaucoup de features (561). D'emblée, en observant ce qu'elles désignent, on peut se dire qu'il y a une certaine redondance entre toutes ces variables. Dans un premier temps, on va effectuer une modélisation "brute" sans se soucier de nettoyer le jeu de données.

Dans un second temps, on va utiliser cette première modélisation pour mieux comprendre le dataset et ainsi effectuer une seconde modélisation plus efficace en éliminant des variables peu importantes.

In [20]:
# Importations des modules/fonctions
import pandas as pd
%pylab inline

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

Populating the interactive namespace from numpy and matplotlib


### 1. Modélisation 'brute' sans features engineering

In [5]:
# Recuperation des données
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [11]:
# check
train.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,STANDING


On va d'abord éliminer les features redondantes (intuitivement, les coordonnées polaires et cartésiennes doivent être corrélées par exemple ... ) Une première manière de faire serait de réfléchir et se renseigner sur le domaine d'études en question pour pouvoir éliminer des variables qui transmettent des informations similaires où n'influencent pas ou très peu la prédiction que l'on veut effectuer.

La seconde manière est d'utiliser justement une forêt aléatoire (!) de laquelle on va extraire l'importance des features qui la constituent, et ainsi déterminer quelles sont les features les plus importantes à partir de ça.

In [16]:
# Recuperation features/target
X_train = train.iloc[:, :-2]
y_train = train['Activity']

X_test = test.iloc[:, :-2]
y_test = test['Activity']

In [19]:
# Modelisation via le forêt aléatoire avec 1000 arbres
rfc = RandomForestClassifier(n_estimators=1000)

#fit
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

Ok, modèle entrainé. 
### IMPORTANT --> RUNTIME
#### En présence d'un dataset volumineux, on peut effectuer une parallélisation des calculs sur plusieurs CPU (avec l'argument  n_jobs  dans scikit-learn).

Voyons maintenant les performances de ce 1er modèle :

In [24]:
print("accuracy %.2f" %rfc.score(X_test, y_test))

accuracy 0.93


Pas mal déjà 😎  !

### 2. Modélisation avec features engineering

#### Sélection des features

Pour cela on va utiliser la fonction  SelectFromModel  qui utilise la propriété du modèle qu'on vient de créer  model.feature_importances_  qui permet d'évaluer l'importance relative des features fournies à la base (sur une échelle de 0 à 1)

Intuitivement, cette importance est calculée en considérant que plus une feature est haute, plus elle contribue à une fraction plus élevée du jeu de donnée d'entraînement et donc des données au global. On considère donc qu'elle a plus d'importance que les features plus bas dans l'arbre. Cette fraction est utilisée comme estimateur de l'importance de la feature dans cet arbre, qu'on peut ensuite généraliser à la forêt entière.

In [43]:
# visualisation de l'importance des features (sur une echelle de 0 à 1) --> 10 premieres valeurs par ex
rfc.feature_importances_[:10]

array([0.00024989, 0.00027238, 0.00017986, 0.00671254, 0.00043755,
       0.00048194, 0.00385255, 0.00049702, 0.00040718, 0.01136162])

In [62]:
# pour take les features importants du model choisi avec un seuil : thresholds
from sklearn.feature_selection import SelectFromModel

select = SelectFromModel(rfc, prefit=True, threshold=0.003)

# on recupère les nouveaux jeu de features d'entrainement et de test
X_train2 = select.transform(X_train)
X_test2 = select.transform(X_test)

# check du nombre de features selectionnés
print(X_train2.shape)

(7352, 84)


On a divisé par 5 le nombre de features utilisées, pas mal mais voyons si les performances restent similaires. À l'aide de l'argument threshold, on peut choisir le seuil d'importance que l'on souhaite pour les features à sélectionner.  On calcule en même temps le gain de temps car c'est ce qui nous intéresse principalement dans l'amélioration des performances.

In [63]:
import timeit #pour get les temps d'execution

# construction d'un second rfc avec des paramètres differents
rfc2 = RandomForestClassifier(n_estimators=500)

start_time = timeit.default_timer()

# fit sur le jeu X_train2 avec moins de features
rfc2 = rfc2.fit(X_train2, y_train)

elapsed = timeit.default_timer() - start_time

print("accuracy {:.2f} time {:.2f}s".format(rfc2.score(X_test2,y_test), elapsed))

#pred = rfc2.predict(X_test2)
#accuracy = accuracy_score(y_test, pred)

accuracy 0.89 time 16.74s


Et pour le 1er modèle avec toutes les features

In [64]:
import timeit #pour get les temps d'execution

# construction d'un second rfc avec des paramètres differents
rfc2 = RandomForestClassifier(n_estimators=500)

start_time = timeit.default_timer()

# fit sur le jeu X_train2 avec moins de features
rfc = rfc.fit(X_train, y_train)

elapsed = timeit.default_timer() - start_time

print("accuracy {:.2f} time {:.2f}s".format(rfc.score(X_test,y_test), elapsed))

accuracy 0.93 time 98.52s


On voit que le modèle avec moins de features (rfc 2) a permis de diviser le temps de calcul par plus de 5 sans trop perdre en performance !!

### Conclusion
On peut continuer à bidouiller afin d'améliorer les performances du modèle finale en modifiant les différents hyperparamètres de contrôle et trouver la bonne balance entre performances de classification / temps de calcul. Bien sûr le ratio souhaité dépendra de la problématique : est-ce qu'on veut une très bonne performance et peu importe le temps de calcul, ou bien un temps de calcul le plus rapide possible mais avec une performance minimum etc

---